In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
merged_1 = orders.merge(users, on="user_id", how="left")
final_df = merged_1.merge(restaurants, on="restaurant_id", how="left")

final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
final_df.shape

(10000, 12)

In [7]:
final_df["membership"].value_counts()

,count
membership,
Regular,5013
Gold,4987


In [8]:
final_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)

,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Pune,1924797.93
Hyderabad,1889366.58


 SECTION 1 : MCQ

In [9]:
# q1

final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [10]:
#q2

final_df.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [11]:
#q3

final_df.groupby("user_id")["total_amount"] \
    .sum() \
    .reset_index() \
    .query("total_amount > 1000") \
    .shape[0]

2544

In [12]:
#q4

final_df.groupby("rating")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
rating,
4.8,657707.71
4.6,495867.97
3.2,490913.01
4.5,479047.03
4.9,467467.09
3.8,466878.69
3.1,443863.92
4.2,423185.06
4.7,416301.51


In [13]:
#q5

final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [14]:
#q6

final_df.groupby("cuisine")["restaurant_id"] \
    .nunique() \
    .sort_values()

,restaurant_id
cuisine,
Chinese,120
Indian,126
Italian,126
Mexican,128


In [15]:
#q7

(final_df["membership"] == "Gold").mean() * 100

np.float64(49.87)

In [16]:
#q8

final_df.groupby("restaurant_name_x") \
    .agg(
        orders=("order_id", "count"),
        avg_value=("total_amount", "mean")
    ) \
    .query("orders < 20") \
    .sort_values("avg_value", ascending=False)

,orders,avg_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [17]:
#q9

final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [18]:
#q10

final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


SECTION -2 : NUMERICAL

In [19]:
#q1

final_df[final_df["membership"] == "Gold"].shape[0]

4987

In [20]:
#q2

final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()

np.float64(1889366.58)

In [21]:
#q3

final_df["user_id"].nunique()

2883

In [22]:
#q4

final_df[final_df["membership"] == "Gold"]["total_amount"].mean()

np.float64(797.1455564467616)

In [23]:
#q5

final_df[final_df["rating"] > 4.5].shape[0]

2772

In [24]:
#q6

gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == gold_city)
].shape[0]

1337